# Portfolio Optimisation with Quantum and Classical Approaches

This notebook explores portfolio optimisation using both binary and continuous weight approaches. We aim to determine the optimal asset allocation in a portfolio to balance return and risk, leveraging both classical and quantum algorithms for comparison.


### Objectives

-   Binary Weight Optimisation:
        Here, asset weights are constrained to binary values (0 or 1), where each selected asset is either included in the portfolio or excluded.
        We implement:
    - Classical Optimisation:  Using integer programming with cvxpy, enforcing binary constraints to select the optimal assets under a fixed budget constraint. Additionally, we use NumPyMinimumEigensolver, a classical eigensolver, to solve the Ising representation of the problem.
    - Quantum Optimisation: Using quantum-inspired solvers like QUBO and Ising formulations to handle binary constraints, leveraging algorithms such as SamplingVQE.

- Continuous Weight Optimisation:
        In this approach, asset weights are continuous, summing to 1, to represent fractional investments in each asset.
        We implement:
    - Classical Optimisation: Using scipy.optimize.minimize (SLSQP method) to perform continuous optimisation, with constraints directly handled to ensure that allocations are fractional and meet budget constraints.
    - Quantum Optimisation: Leveraging QUBO-based quantum approaches to handle the continuous optimisation problem.


In [1]:
from qiskit.circuit.library import TwoLocal
from qiskit.circuit.library import EfficientSU2

from qiskit.result import QuasiDistribution
from qiskit_aer.primitives import Sampler
from qiskit_algorithms import NumPyMinimumEigensolver, QAOA, SamplingVQE, VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_optimization.algorithms import MinimumEigenOptimizer
import numpy as np
import matplotlib.pyplot as plt
import datetime
import datetime as dt
import pandas as pd
import yfinance as yf
from qiskit.primitives import Sampler, Estimator
from scipy.optimize import minimize
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_algorithms.utils import algorithm_globals

algorithm_globals.random_seed = 1234


In this exercise we will only do portfolio optimisation for the energy market, so we will need to collect data on various energy-related stocks and indices:

In [2]:
start_date = dt.datetime(2023, 1, 1)
end_date =  dt.datetime(2024, 1, 1)

def get_data(ticker):
    return pd.DataFrame(yf.download(ticker, start=start_date, end=end_date))['Adj Close']

# Energy Stocks
energy_stocks = ['ENPH', 'FSLR', 'SHEL','XOM' ] #
stock_data_direct = {stock: get_data(stock) for stock in energy_stocks}

stock_data_direct = pd.DataFrame.from_dict(stock_data_direct)
mu = stock_data_direct.pct_change(1).dropna().mean().values #mean return
sigma  = (stock_data_direct/stock_data_direct.shift(1)).dropna().cov().values #covariance return

num_assets = len(stock_data_direct.columns)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
q = 0.5  # set risk factor
budget = num_assets // 2  # set budget

portfolio = PortfolioOptimization(
    expected_returns=mu, covariances=sigma, risk_factor=q, budget=budget
    )
qp = portfolio.to_quadratic_program()

conv = QuadraticProgramToQubo()

qubit_op = conv.convert(qp) #Here I convert to QUBO and remove the constraints
op, offset = qubit_op.to_ising()

In [4]:
def print_result(result):
    selection = result.x
    value = result.fval
    print("Optimal: selection {}, value {:.6f}".format(selection, value))

    eigenstate = result.min_eigen_solver_result.eigenstate
    probabilities = (
        eigenstate.binary_probabilities()
        if isinstance(eigenstate, QuasiDistribution)
        else {k: np.abs(v) ** 2 for k, v in eigenstate.to_dict().items()}
    )
    print("\n----------------- Full result ---------------------")
    print("selection\tvalue\t\tprobability")
    print("---------------------------------------------------")
    probabilities = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)

    for k, v in probabilities:
        x = np.array([int(i) for i in list(reversed(k))])
        value = portfolio.to_quadratic_program().objective.evaluate(x)
        print("%10s\t%.6f\t\t%.6f" % (x, value, v))



# Binary Optimisation

NumPyMinimumEigensolver using QUBO representation

In [5]:
exact_mes = NumPyMinimumEigensolver()
exact_eigensolver = MinimumEigenOptimizer(exact_mes)
result = exact_eigensolver.solve(qp)

#print_result(result)
print("Numpy Minimum Eigensolver Optimal Portfolio Allocation:", result.x)
print("Numpy Minimum Eigensolver Optimal Portfolio Value:", result.fval)

Numpy Minimum Eigensolver Optimal Portfolio Allocation: [0. 1. 1. 0.]
Numpy Minimum Eigensolver Optimal Portfolio Value: -0.0013783205095481562


## Solution using SamplingVQE

In [6]:
optimizer = COBYLA(maxiter=500)
ansatz = TwoLocal(num_assets, "ry", "cz", reps=3, entanglement="full")
svqe_mes = SamplingVQE(sampler=Sampler(), ansatz=ansatz, optimizer=optimizer)
svqe = MinimumEigenOptimizer(svqe_mes)
svqe_result = svqe.solve(qp)
#print(svqe_result.prettyprint())
#print('-------------------')
#print_result(svqe_result)

print("SamplingVQE Optimal Portfolio Allocation:", svqe_result.x)
print("SamplingVQE Optimal Portfolio Value:", svqe_result.fval)

SamplingVQE Optimal Portfolio Allocation: [0. 1. 1. 0.]
SamplingVQE Optimal Portfolio Value: -0.0013783205095481562


## Classic binary optimisation, using cvxpy

In [7]:
import cvxpy as cp
# Define the binary optimization variables
x = cp.Variable(num_assets, boolean=True)
# Define the portfolio objective
objective = cp.Maximize(mu @ x - q * cp.quad_form(x, sigma))


# Define the constraints
constraints = [cp.sum(x) == budget] 

problem = cp.Problem(objective, constraints)
problem.solve()

print("Binary Portfolio Allocation:", x.value)
print("Portfolio Objective Value:", - problem.value)

Restricted license - for non-production use only - expires 2025-11-24
Binary Portfolio Allocation: [0. 1. 1. 0.]
Portfolio Objective Value: -0.0013783205095481562


# Continuous Optimisation
## Classic optimisation using constrained minimisation

In [8]:
def mean_variance_optimization(cov_matrix, expected_returns, lambda_risk):
    n_assets = len(expected_returns)    
    # Define the objective function (to minimize)
    def objective(weights):
        portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
        return lambda_risk * portfolio_variance - np.dot(weights, expected_returns)
    
    # Constraints: weights sum to 1
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    # Bounds: no short selling (weights between 0 and 1)
    bounds = tuple((0, 1) for _ in range(n_assets))
    # Initial guess 
    initial_weights = np.array(n_assets * [1. / n_assets])    
    result = minimize(objective, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    
    return result.x, result.fun 

# Run the classical optimization
optimal_weights_classical, value_objective = mean_variance_optimization(sigma, mu, q)

print("Optimal Portfolio Weights (Classical):", optimal_weights_classical)
print(f"Classical objective value: {value_objective}")



Optimal Portfolio Weights (Classical): [0.         0.36498759 0.37109238 0.26392003]
Classical objective value: -0.0006492673615752576


## VQE optimisation using constrained minimisation
(in development)